# Preprocessing

The binance dataset will be analyzed in this notebook to correctly normalize and preprocess the data

The code in this notebook will then be used in the crypto_price_analysis/load.py such that it can be reused later

***Warning: this cell may take a while to run as it has to dowload hundreds of files from the binance servers***

In [31]:
%load_ext autoreload
%autoreload 2

import sys
import os
sys.path.append(os.path.abspath('..'))

from pathlib import Path
from crypto_price_analysis.load import load_binance_dataset


data = download_binance_dataset(Path('../data'), 'ETHUSDT', '1m')
data

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


,open,high,low,close,volume,count,taker_buy_volume
open_time,,,,,,,
2017-08-17 04:00:00,301.13,301.13,301.13,301.13,0.42643,2,0.42643
2017-08-17 04:01:00,301.13,301.13,301.13,301.13,2.75787,4,2.75787
2017-08-17 04:02:00,300.00,300.00,300.00,300.00,0.09930,2,0.09930
2017-08-17 04:03:00,300.00,300.00,300.00,300.00,0.31389,3,0.00000
2017-08-17 04:04:00,301.13,301.13,301.13,301.13,0.23202,1,0.23202
...,...,...,...,...,...,...,...
2024-07-17 23:55:00,3387.15,3387.48,3386.89,3386.90,32.27540,194,15.87380
2024-07-17 23:56:00,3386.89,3393.00,3386.89,3392.75,185.75390,658,132.00590
2024-07-17 23:57:00,3392.75,3392.76,3388.70,3389.47,74.63250,382,34.08340


In [32]:
data.describe()

,open,high,low,close,volume,count,taker_buy_volume
count,3.630078e+06,3.630078e+06,3.630078e+06,3.630078e+06,3.630078e+06,3.630078e+06,3.630078e+06
mean,1.356906e+03,1.357799e+03,1.356007e+03,1.356905e+03,3.704932e+02,4.080876e+02,1.851769e+02
std,1.196566e+03,1.197289e+03,1.195840e+03,1.196567e+03,6.969689e+02,6.980224e+02,3.637699e+02
min,8.202000e+01,8.208000e+01,8.179000e+01,8.203000e+01,0.000000e+00,0.000000e+00,0.000000e+00
25%,2.516000e+02,2.518600e+02,2.513500e+02,2.516100e+02,7.104940e+01,8.000000e+01,3.090094e+01
50%,1.184125e+03,1.185020e+03,1.183050e+03,1.184120e+03,1.735654e+02,2.150000e+02,8.246069e+01
75%,2.050990e+03,2.051930e+03,2.050020e+03,2.050990e+03,3.979017e+02,4.800000e+02,1.970472e+02
max,4.865220e+03,4.868000e+03,4.861380e+03,4.865220e+03,4.013161e+04,6.375100e+04,2.374347e+04


First, lets use linear interpolation to remove the 0 values as it will cause problem when taking the log later

In [33]:
# interpolate data based on the datetime index
data = data.mask(data == 0).interpolate('time')

In [34]:
from datetime import timedelta
import pandas as pd

(pd.Series(data.index.values).diff() != timedelta(minutes=1)).sum()

36

As we can see, some chunks are missing in the dataset, lets use linear interpolation again for the gaps small enough

In [35]:
import pandas as pd
import numpy as np

from datetime import timedelta

max_gap_size = 60 # the maximum missing values we are willing to interpolate

# as_freq adds the missing rows filled with Nan
data = data.asfreq(timedelta(minutes=1)).interpolate('linear', limit=max_gap_size)

data

,open,high,low,close,volume,count,taker_buy_volume
open_time,,,,,,,
2017-08-17 04:00:00,301.13,301.13,301.13,301.13,0.42643,2.0,0.42643
2017-08-17 04:01:00,301.13,301.13,301.13,301.13,2.75787,4.0,2.75787
2017-08-17 04:02:00,300.00,300.00,300.00,300.00,0.09930,2.0,0.09930
2017-08-17 04:03:00,300.00,300.00,300.00,300.00,0.31389,3.0,0.16566
2017-08-17 04:04:00,301.13,301.13,301.13,301.13,0.23202,1.0,0.23202
...,...,...,...,...,...,...,...
2024-07-17 23:55:00,3387.15,3387.48,3386.89,3386.90,32.27540,194.0,15.87380
2024-07-17 23:56:00,3386.89,3393.00,3386.89,3392.75,185.75390,658.0,132.00590
2024-07-17 23:57:00,3392.75,3392.76,3388.70,3389.47,74.63250,382.0,34.08340


The sequence used by the neural net should not overlap samples that are not consecutives, let's separate our dataset into chunks

In [36]:

chunks_index = data.isna().any(axis=1).diff().fillna(0).cumsum()
# chunks with odd index are filled with NaN values
chunks_index

open_time
2017-08-17 04:00:00     0
2017-08-17 04:01:00     0
2017-08-17 04:02:00     0
2017-08-17 04:03:00     0
2017-08-17 04:04:00     0
                       ..
2024-07-17 23:55:00    56
2024-07-17 23:56:00    56
2024-07-17 23:57:00    56
2024-07-17 23:58:00    56
2024-07-17 23:59:00    56
Freq: min, Length: 3638640, dtype: object

In [37]:
from itertools import islice

chunks: list[pd.DataFrame] = []

groups = data.groupby(chunks_index)
for g in islice(groups.groups, 0, None, 2):
	chunk = groups.get_group(g).dropna()
	if chunk.shape[0] > max_gap_size:
		chunks.append(chunk)


To represent the featues we are going to apply the function $\log(1+rate)$ where $rate=\frac{p_{i}}{p_{i-1}}$, this will give our features a normal distibution if we assume they are distributed log normally

In [38]:

for chunk in chunks:
	# get log pct change for the price columns
	chunk[:] = np.log1p(chunk.pct_change())
	chunk.dropna(inplace=True)


concatenated_chunks = pd.concat(chunks) 
mean = concatenated_chunks.mean()
std = concatenated_chunks.std()


for chunk in chunks:
	chunk = (chunk - mean) / std

In [40]:
mean, std

(open                5.144323e-07
 high                4.885575e-07
 low                 5.389192e-07
 close               5.121573e-07
 volume             -5.742760e-06
 count              -4.827115e-06
 taker_buy_volume   -4.996952e-06
 dtype: float64,
 open                0.001545
 high                0.001346
 low                 0.001508
 close               0.001478
 volume              0.929668
 count               0.533837
 taker_buy_volume    1.113490
 dtype: float64)